In [1]:
import numpy as np
import pandas as pd

data=pd.read_csv('binaryclassification-predicting-defaulters-dataset.csv')



# Information About Dataset

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1816 entries, 0 to 1815
Data columns (total 8 columns):
Response          1816 non-null object
Age               1816 non-null int64
Job               1816 non-null object
Marital Status    1816 non-null object
Education         1816 non-null object
balance           1816 non-null int64
housing           1816 non-null object
loan              1816 non-null object
dtypes: int64(2), object(6)
memory usage: 113.6+ KB


# Head fo Dataset

In [3]:
data.head(5)

,Response,Age,Job,Marital Status,Education,balance,housing,loan
0,yes,59,retired,married,secondary,49,no,yes
1,yes,31,services,married,tertiary,90,no,yes
2,yes,47,management,married,primary,145,no,yes
3,yes,45,unemployed,divorced,primary,249,yes,yes
4,yes,36,management,single,tertiary,559,yes,no


# Check The Dataset Contain NaN values or Not

In [6]:
data.isnull().sum()

Response          0
Age               0
Job               0
Marital Status    0
Education         0
balance           0
housing           0
loan              0
dtype: int64

# Convert the object datatype into the numerical datatype

In [7]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['Response']=le.fit_transform(data['Response'])
data['Job']=le.fit_transform(data['Job'])
data['Marital Status']=le.fit_transform(data['Marital Status'])
data['Education']=le.fit_transform(data['Education'])
data['housing']=le.fit_transform(data['housing'])
data['loan']=le.fit_transform(data['loan'])

In [40]:
data.head(7)

,Response,Age,Job,Marital Status,Education,balance,housing,loan
0,1,59,5,1,1,49,0,1
1,1,31,7,1,2,90,0,1
2,1,47,4,1,0,145,0,1
3,1,45,10,0,0,249,1,1
4,1,36,4,2,2,559,1,0
5,1,48,7,1,0,583,1,0
6,1,49,9,0,1,757,1,1


# Feature Selection

In [9]:
from sklearn.feature_selection import chi2,SelectKBest
len(data.columns)
m=data.iloc[:,0:6]
n=data.iloc[:,7]

bestfeature=SelectKBest(score_func=chi2,k=3)
fit=bestfeature.fit(m,n)

dfscore=pd.DataFrame(fit.scores_)
dfcolumn=pd.DataFrame(m.columns)
featurescore=pd.concat([dfcolumn,dfscore],axis=1)
featurescore.columns=('Var','Score')
print(featurescore.nlargest(3,'Score'))

        Var        Score
5   balance  3548.866995
0  Response    80.012403
1       Age     7.087773


# Create the input and output variable

In [15]:
l=['Response','Age','balance']
X=data[l]
y=data['loan']


# Split the Train and Test 

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=5)


# Model Selection 

In [17]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier()

from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()

from sklearn.svm import SVC
sc=SVC()

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()


# Use Cross Validation for model selection

In [18]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(lr,X,y,cv=10,scoring='accuracy').mean())


from sklearn.model_selection import cross_val_score
print(cross_val_score(dtc,X,y,cv=10,scoring='accuracy').mean())

from sklearn.model_selection import cross_val_score
print(cross_val_score(rfc,X,y,cv=10,scoring='accuracy').mean())

from sklearn.model_selection import cross_val_score
print(cross_val_score(sc,X,y,cv=10,scoring='accuracy').mean())

from sklearn.model_selection import cross_val_score
print(cross_val_score(knn,X,y,cv=10,scoring='accuracy').mean())


0.7626677190213103
0.6739299374658492
0.7074615991743064
0.7637666201202113
0.7130805658429968


# Number of count in the Output Variable 

In [50]:
data.groupby('loan').size()

loan
0    1387
1     429
dtype: int64

# Accuracy of the SVM is high

In [19]:
sc.fit(X_train,y_train)
print("Accuracy of model using Train Dataset :",sc.score(X_train,y_train))
print("Accuracy of model using Test Dataset :",sc.score(X_test,y_test))


Accuracy of model using Train Dataset : 0.7555066079295154
Accuracy of model using Test Dataset : 0.7929515418502202


In [51]:
sc.predict([[1,59,49]])

array([0])

In [52]:
sc.predict([[1,36,559]])

array([0])

In [25]:
sc.predict([[1,59,49]])

array([0])

Dataset is imbalance(Number of Ones is less than the number of Zeros) so the

Predication is wrong 

# Balance Dataset

In [30]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_sample(X_train.astype('float'),y_train)
from collections import Counter

# Check The Number of counts

In [31]:
print("Before SMOTE :" , Counter(y_train))
print("After SMOTE :" , Counter(y_train_smote))

Before SMOTE : Counter({0: 1027, 1: 335})
After SMOTE : Counter({0: 1027, 1: 1027})


# Fit the Decision Tree Model for the Balance Dataset

In [42]:
dtc.fit(X_train_smote, y_train_smote)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [46]:
dtc.predict([[1,59,49]])

array([1])

In [47]:
dtc.predict([[1,48,583]])

array([0])

In [48]:
dtc.predict([[1,31,90]])

array([1])

In [49]:
dtc.predict([[1,36,559]])

array([0])